#### Lab 7 
#### Rouge Metrics

In [1]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
import numpy as np
from fuzzywuzzy import fuzz 
import ipywidgets as widgets
import pprint
from ipywidgets import interact, interact_manual
import re
__PATH__ = "./data.csv"

/home/sultan/miniconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


/opt/tljh/user/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [6]:
#import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/jupyter-
[nltk_data]     alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter-
[nltk_data]     alex/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
df = pd.read_csv(__PATH__,sep=";",header=0)

In [3]:
df

,id,updatedDate,publishedDate,title,summary,authors,category,metaData,downloadLink,filePath
0,http://arxiv.org/abs/1407.6950v1,2014-07-24T16:56:39Z,2014-07-24T16:56:39Z,"How,whenAndHowMuchACardDeckIsWellShuffled.pdf",The Thesis Consider The Mixing Of Few 3 4 ...,Benjamin Isac Fargion,cs.DM,"Italian Thesis In Engeenering Computer, 26 Feb...",http://arxiv.org/pdf/1407.6950v1.pdf,"./files/How,whenAndHowMuchACardDeckIsWellShuff..."
1,http://arxiv.org/abs/0907.0618v1,2009-07-03T12:35:10Z,2009-07-03T12:35:10Z,QuantumIsometryGroups.pdf,This Thesis Contains The Formulation And Com...,Jyotishman Bhowmick,math.OA,Thesis,http://arxiv.org/pdf/0907.0618v1.pdf,./files/QuantumIsometryGroups.pdf
2,http://arxiv.org/abs/1806.09601v2,2018-07-14T17:06:27Z,2018-06-25T17:55:59Z,ComputationAndBoundingOfFolkmanNumbers.pdf,Phd Thesis Under The Supervision Of Professo...,Aleksandar Bikov,math.CO,PhD Thesis,http://arxiv.org/pdf/1806.09601v2.pdf,./files/ComputationAndBoundingOfFolkmanNumbers...
3,http://arxiv.org/abs/1905.03014v1,2019-05-08T11:47:34Z,2019-05-08T11:47:34Z,OnChurch'sThesisInCubicalAssemblies.pdf,"We Show That Church's Thesis, The Axiom Stat...","Andrew Swan, Taichi Uemura,",math.LO,0,http://arxiv.org/pdf/1905.03014v1.pdf,./files/OnChurch'sThesisInCubicalAssemblies.pdf
4,http://arxiv.org/abs/1901.04911v1,2019-01-15T16:24:07Z,2019-01-15T16:24:07Z,UnconstrainedChurchTuringThesisCannotPossiblyB...,The Church Turing Thesis Asserts That If A P...,Yuri Gurevich,cs.LO,0,http://arxiv.org/pdf/1901.04911v1.pdf,./files/UnconstrainedChurchTuringThesisCannotP...
...,...,...,...,...,...,...,...,...,...,...
991,http://arxiv.org/abs/0709.0497v1,2007-09-04T18:40:50Z,2007-09-04T18:40:50Z,HardSpectatorInteractionsInBToPiPiAtOrderAlpha...,In The Present Thesis I Discuss The Hard Spe...,Volker Pilipp,hep-ph,PhD thesis,http://arxiv.org/pdf/0709.0497v1.pdf,./files/HardSpectatorInteractionsInBToPiPiAtOr...
992,http://arxiv.org/abs/0709.2936v1,2007-09-18T23:56:17Z,2007-09-18T23:56:17Z,BayesianClassificationAndRegressionWithHighDim...,This Thesis Responds To The Challenges Of Us...,Longhai Li,stat.ML,"PhD Thesis Submitted to University of Toronto,...",http://arxiv.org/pdf/0709.2936v1.pdf,./files/BayesianClassificationAndRegressionWit...
993,http://arxiv.org/abs/0710.1790v1,2007-10-09T15:20:08Z,2007-10-09T15:20:08Z,StaticAndDynamicPropertiesOfHadronicSystemsWit...,This Thesis Has Been Devoted To The Study Of...,J. M. Verde-Velasco,hep-ph,"PhD thesis, Supervisors: E. Hernandez, J. Niev...",http://arxiv.org/pdf/0710.1790v1.pdf,./files/StaticAndDynamicPropertiesOfHadronicSy...
994,http://arxiv.org/abs/0711.0873v2,2007-11-11T19:21:04Z,2007-11-06T14:13:50Z,AConformalApproachToNumericalCalculationsOfAsy...,This Thesis Is Concerned With The Developmen...,Anıl Zenginoğlu,gr-qc,"PhD thesis, Max-Planck Institute for Gravitati...",http://arxiv.org/pdf/0711.0873v2.pdf,./files/AConformalApproachToNumericalCalculati...


In [4]:
df.shape

(996, 10)

#### Preprocessing the title to list of tokens

In [4]:
titles = list(df['title'].apply(
    lambda t : 
        tuple(
            filter(lambda e:not e in stopwords.words('english'),
                map(lambda e:e.lower(),
                       re.findall('([A-Z]{1}[a-z]+)',t.replace('.pdf','')))
                )
            )
        )
    )

In [6]:
titles[:20]

[('much', 'card', 'deck', 'well', 'shuffled'),
 ('quantum', 'isometry', 'groups'),
 ('computation', 'bounding', 'folkman', 'numbers'),
 ('church', 'thesis', 'cubical', 'assemblies'),
 ('unconstrained', 'church', 'turing', 'thesis', 'cannot', 'possibly', 'true'),
 ('algebraic',
  'relaxations',
  'hardness',
  'results',
  'polynomial',
  'optimization',
  'lyapunov',
  'analysis'),
 ('resolving',
  'complexity',
  'fundamental',
  'problems',
  'computational',
  'social',
  'choice'),
 ('pa',
  'instantiationally',
  'complete',
  'algorithmically',
  'incomplete',
  'alternative',
  'interpretation',
  'goedelian',
  'incompleteness',
  'church',
  'thesis',
  'links',
  'formal',
  'logic',
  'computability'),
 ('numerical',
  'modeling',
  'fluid',
  'flow',
  'porous',
  'media',
  'modelowanie',
  'numeryczne',
  'transportu',
  'plynow',
  'przez',
  'osrodki',
  'porowate'),
 ('threebranes', 'theory'),
 ('spin',
  'torque',
  'nano',
  'oscillators',
  'memristors',
  'multi',


In [5]:
res = {}
for title in titles[:20]:
    synsets = {}
    for word in title:
        synsets[word]=[synset for synset in wn.synsets(word)]
    res[title] = synsets

In [48]:
res

{('much',
  'card',
  'deck',
  'well',
  'shuffled'): {'much': [Synset('much.n.01'),
   Synset('much.a.01'),
   Synset('much.r.01'),
   Synset('much.r.02'),
   Synset('a_lot.r.01'),
   Synset('much.r.04'),
   Synset('much.r.05')], 'card': [Synset('card.n.01'),
   Synset('card.n.02'),
   Synset('card.n.03'),
   Synset('card.n.04'),
   Synset('wag.n.01'),
   Synset('poster.n.01'),
   Synset('calling_card.n.02'),
   Synset('card.n.08'),
   Synset('menu.n.01'),
   Synset('batting_order.n.01'),
   Synset('circuit_board.n.01'),
   Synset('tease.v.07'),
   Synset('card.v.02')], 'deck': [Synset('deck.n.01'),
   Synset('deck.n.02'),
   Synset('pack_of_cards.n.01'),
   Synset('deck.n.04'),
   Synset('deck.v.01'),
   Synset('deck.v.02'),
   Synset('deck.v.03')], 'well': [Synset('well.n.01'),
   Synset('well.n.02'),
   Synset('well.n.03'),
   Synset('well.n.04'),
   Synset('well.n.05'),
   Synset('well.v.01'),
   Synset('well.a.01'),
   Synset('good.s.13'),
   Synset('well.s.03'),
   Synset('well

In [9]:
reslist = list(res.items())
print(len(reslist))
print(reslist[0])

20
(('much', 'card', 'deck', 'well', 'shuffled'), {'much': [Synset('much.n.01'), Synset('much.a.01'), Synset('much.r.01'), Synset('much.r.02'), Synset('a_lot.r.01'), Synset('much.r.04'), Synset('much.r.05')], 'card': [Synset('card.n.01'), Synset('card.n.02'), Synset('card.n.03'), Synset('card.n.04'), Synset('wag.n.01'), Synset('poster.n.01'), Synset('calling_card.n.02'), Synset('card.n.08'), Synset('menu.n.01'), Synset('batting_order.n.01'), Synset('circuit_board.n.01'), Synset('tease.v.07'), Synset('card.v.02')], 'deck': [Synset('deck.n.01'), Synset('deck.n.02'), Synset('pack_of_cards.n.01'), Synset('deck.n.04'), Synset('deck.v.01'), Synset('deck.v.02'), Synset('deck.v.03')], 'well': [Synset('well.n.01'), Synset('well.n.02'), Synset('well.n.03'), Synset('well.n.04'), Synset('well.n.05'), Synset('well.v.01'), Synset('well.a.01'), Synset('good.s.13'), Synset('well.s.03'), Synset('well.r.01'), Synset('well.r.02'), Synset('well.r.03'), Synset('well.r.04'), Synset('well.r.05'), Synset('wel

In [10]:
print(reslist[0][1]['much'])

[Synset('much.n.01'), Synset('much.a.01'), Synset('much.r.01'), Synset('much.r.02'), Synset('a_lot.r.01'), Synset('much.r.04'), Synset('much.r.05')]


In [11]:
print(reslist[0][0])

('much', 'card', 'deck', 'well', 'shuffled')


In [12]:
resset = set(reslist[0][0])
resset

{'card', 'deck', 'much', 'shuffled', 'well'}

In [13]:
for word in resset:
    print(type(word), word)

<class 'str'> card
<class 'str'> much
<class 'str'> deck
<class 'str'> well
<class 'str'> shuffled


In [12]:
shufsynset = wn.synsets('shuffled')
shufsynset

[Synset('shuffle.v.01'), Synset('shuffle.v.02'), Synset('shuffle.v.03')]

In [13]:
synset = wn.synsets('shuffled')[0]
synset.hypernyms()

[Synset('walk.v.01')]

In [14]:
def get_parent_classes(synset):
    hyps = set()
    while True:
        try:
            synset = synset.hypernyms()[-1]
            hyps.add(synset)
        except IndexError:
            break
    return hyps

In [15]:
get_parent_classes(shufsynset[0])

{Synset('travel.v.01'), Synset('walk.v.01')}

In [6]:
def get_2parent_classes(synset):
    hyps = dict()
    level_count = 1
    while level_count < 3:
        try:
            synset = synset.hypernyms()[0]
            hyps[level_count] = synset 
            level_count += 1
        except IndexError:
            break
    return hyps

In [30]:
def extract_hypornyms_en(a):
    hypornyms_en_first = set()
    hypornyms_en_second = set()
    for word in a:
        # find the hypernyms of the word
        # word = wn.synset(word)
        try:
            word_synset0 = wn.synsets(word)[0]
            hyps_buff = get_2parent_classes(word_synset0)
            #print(hyps_buff)
            hypornyms_en_first.add(hyps_buff[1].name().split('.')[0])
            hypornyms_en_second.add(hyps_buff[2].name().split('.')[0])
        except IndexError:
            continue
        except KeyError:
            continue
    return hypornyms_en_first, hypornyms_en_second

In [31]:
extract_hypornyms_en(resset)

({'excavation', 'large_indefinite_quantity', 'paper', 'platform', 'walk'},
 {'artifact',
  'horizontal_surface',
  'indefinite_quantity',
  'material',
  'travel'})

In [32]:
def f1measure(a, b):
    a = set(a)
    b = set(b)
    # missed part
    overlap = set()
    overlap_hyp_cnt = 0
    overlap = a.intersection(b)
    overlap_hyp_cnt = len(overlap) 
    
    recl = overlap_hyp_cnt/len(a)
    prec = overlap_hyp_cnt/len(b)
    
    if len(overlap) == 0:
        return 0, overlap
    else:
        return 2*recl*prec/(recl+prec), overlap

In [19]:
print(reslist[0][0])
print(reslist[1][0])

('much', 'card', 'deck', 'well', 'shuffled')
('quantum', 'isometry', 'groups')


In [20]:
aset = set(reslist[0][0])
bset = set(('much', 'card', 'deck'))

In [21]:
print(aset.intersection(bset))

{'card', 'much', 'deck'}


In [22]:
sss = set(reslist[0][1])
sss

{'card', 'deck', 'much', 'shuffled', 'well'}

In [35]:
for x in sss:
    print(x)

shuffled
deck
card
much
well


In [36]:
'shuffled' in sss

True

In [44]:
ddd = dict(reslist[0][1])
ddd

{'much': [Synset('much.n.01'),
  Synset('much.a.01'),
  Synset('much.r.01'),
  Synset('much.r.02'),
  Synset('a_lot.r.01'),
  Synset('much.r.04'),
  Synset('much.r.05')],
 'card': [Synset('card.n.01'),
  Synset('card.n.02'),
  Synset('card.n.03'),
  Synset('card.n.04'),
  Synset('wag.n.01'),
  Synset('poster.n.01'),
  Synset('calling_card.n.02'),
  Synset('card.n.08'),
  Synset('menu.n.01'),
  Synset('batting_order.n.01'),
  Synset('circuit_board.n.01'),
  Synset('tease.v.07'),
  Synset('card.v.02')],
 'deck': [Synset('deck.n.01'),
  Synset('deck.n.02'),
  Synset('pack_of_cards.n.01'),
  Synset('deck.n.04'),
  Synset('deck.v.01'),
  Synset('deck.v.02'),
  Synset('deck.v.03')],
 'well': [Synset('well.n.01'),
  Synset('well.n.02'),
  Synset('well.n.03'),
  Synset('well.n.04'),
  Synset('well.n.05'),
  Synset('well.v.01'),
  Synset('well.a.01'),
  Synset('good.s.13'),
  Synset('well.s.03'),
  Synset('well.r.01'),
  Synset('well.r.02'),
  Synset('well.r.03'),
  Synset('well.r.04'),
  Synse

In [43]:
ddd['shuffled']

[Synset('shuffle.v.01'), Synset('shuffle.v.02'), Synset('shuffle.v.03')]

In [23]:
f1measure(reslist[0][0], reslist[1][0])

(0, set())

In [36]:
lang = 'eng'

    
#def distance(a,b):
    ### Put your code here
    ### В переменной synsets помимо нормализованного заголовка хранятся синсеты для каждого токена из заголовка.
    ### у синсетов есть гиперонимы про то как из брать здесь
    ### Если у токенов заголовков есть общие гиперонимы тогда заголовки связаны несмотря на то что слова разные
    
    #return float((100-fuzz.ratio(a,b))/100)

    
def distance(a,b):
    ### Put your code here
    ### В переменной synsets помимо нормализованного заголовка хранятся синсеты для каждого токена из заголовка.
    ### у синсетов есть гиперонимы про то как из брать здесь
    ### Если у токенов заголовков есть общие гиперонимы тогда заголовки связаны несмотря на то что слова разные
    
    a = set(a) 
    b = set(b) 
    f1score, overlap = f1measure(a, b)
    a = a - overlap
    b = b - overlap
    
    if len(a) == 0 or len(b) == 0:
        return 1.0 - f1score
    
    buff_a = {}
    buff_b = {}
    
    if lang == 'eng':
        buff_a_first, buff_a_second = extract_hypornyms_en(a)
        buff_b_first, buff_b_second = extract_hypornyms_en(b)
        
    overlap_hyp_cnt = 0
    print('buff_a_first',buff_a_first)
    print('buff_b_first',buff_b_first)
    print('buff_a_second',buff_a_second)
    print('buff_b_second',buff_b_second)
    #for word in a:
    #    for wordb in b:
    #        if len(buff_a[word].intersection(buff_b[wordb])) > 0:
    #            overlap_hyp_cnt += 1
    
    if len(buff_a_first.intersection(buff_b_first)) > 0:
        overlap_hyp_cnt += 1
    
    if len(buff_a_second.intersection(buff_b_second)) > 0:
        overlap_hyp_cnt += 0.5
    
    
    
    recl_hyp = overlap_hyp_cnt/len(a)
    #print('recl_hyp',recl_hyp)
    prec_hyp = overlap_hyp_cnt/len(b)
    #print('prec_hyp',prec_hyp)
    f1score_hyp = 2*recl_hyp/(recl_hyp + prec_hyp) if overlap_hyp_cnt > 0 else 0
    # print('f1score_hyp',f1score_hyp)
    f1res = (2*f1score+f1score_hyp)/3
    #print('f1res',f1res,'\n')
    return (1.0 - f1res)

buff = list(res.items())
dist = np.zeros((len(buff),len(buff)))
for lli,ll in enumerate(buff):
    for rri,rr in enumerate(buff):
        dist[lli,rri] = distance(ll[0], rr[0])
        

buff_a_first {'paper', 'walk', 'excavation', 'large_indefinite_quantity', 'platform'}
buff_b_first {'growth_rate', 'quantity', 'abstraction'}
buff_a_second {'horizontal_surface', 'travel', 'artifact', 'material', 'indefinite_quantity'}
buff_b_second {'entity', 'concept', 'rate'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'paper', 'walk', 'excavation', 'large_indefinite_quantity', 'platform'}
buff_b_first {'move', 'procedure'}
buff_a_second {'horizontal_surface', 'travel', 'artifact', 'material', 'indefinite_quantity'}
buff_b_second {'activity'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'paper', 'walk', 'excavation', 'large_indefinite_quantity', 'platform'}
buff_b_first {'religion', 'premise', 'machine'}
buff_a_second {'horizontal_surface', 'travel', 'artifact', 'material', 'indefinite_quantity'}
buff_b_second {'institution', 'postulate', 'device'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'paper', 'walk', 'excavatio

buff_b_first {'explanation'}
buff_a_second {'activity'}
buff_b_second {'thinking'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'move', 'procedure'}
buff_b_first {'rotation', 'generator', 'engineering', 'travel', 'force'}
buff_a_second {'activity'}
buff_b_second {'turning', 'discipline', 'electronic_device', 'physical_phenomenon'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'move', 'procedure'}
buff_b_first {'relation', 'quantity'}
buff_a_second {'activity'}
buff_b_second {'concept', 'abstraction'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'move', 'procedure'}
buff_b_first {'idea', 'delay', 'music_genre'}
buff_a_second {'activity'}
buff_b_second {'expressive_style', 'pause', 'content'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'move', 'procedure'}
buff_b_first {'computer_operation', 'activity', 'commercial_document', 'message', 'sculpture'}
buff_a_second {'activity'}
buff_b_second {'operation', '

buff_a_first {'alignment', 'religion', 'premise'}
buff_b_first {'rotation', 'generator', 'engineering', 'travel', 'force'}
buff_a_second {'institution', 'postulate', 'placement'}
buff_b_second {'turning', 'discipline', 'electronic_device', 'physical_phenomenon'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'alignment', 'religion', 'premise'}
buff_b_first {'relation', 'quantity'}
buff_a_second {'institution', 'postulate', 'placement'}
buff_b_second {'concept', 'abstraction'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'alignment', 'religion', 'premise'}
buff_b_first {'idea', 'delay', 'music_genre'}
buff_a_second {'institution', 'postulate', 'placement'}
buff_b_second {'expressive_style', 'pause', 'content'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'alignment', 'religion', 'premise'}
buff_b_first {'computer_operation', 'activity', 'commercial_document', 'message', 'sculpture'}
buff_a_second {'institution', 'postulate', '

prec_hyp 0.1
f1score_hyp 0.5263157894736841
f1res 0.23893065998329152 

buff_a_first {'representation', 'philosophy', 'religion', 'decision_making', 'dance', 'golf_course', 'father', 'integrity', 'premise', 'end'}
buff_b_first {'computer_operation', 'activity', 'commercial_document', 'message', 'sculpture'}
buff_a_second {'party', 'higher_cognitive_process', 'content', 'course', 'institution', 'state', 'change', 'parent', 'humanistic_discipline', 'postulate'}
buff_b_second {'operation', 'art', 'act', 'document', 'communication'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'representation', 'philosophy', 'religion', 'decision_making', 'dance', 'golf_course', 'father', 'integrity', 'premise', 'end'}
buff_b_first {'procedure', 'use', 'similarity'}
buff_a_second {'party', 'higher_cognitive_process', 'content', 'course', 'institution', 'state', 'change', 'parent', 'humanistic_discipline', 'postulate'}
buff_b_second {'sameness', 'activity'}
recl_hyp 0.0
prec_hyp 0.0
f1s

buff_b_first {'change_of_location', 'sculpture', 'substance', 'instrumentality'}
buff_a_second {'thinking'}
buff_b_second {'plastic_art', 'movement', 'matter', 'artifact'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'explanation'}
buff_b_first {'rotation', 'generator', 'engineering', 'travel', 'force'}
buff_a_second {'thinking'}
buff_b_second {'turning', 'discipline', 'electronic_device', 'physical_phenomenon'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'explanation'}
buff_b_first {'relation', 'quantity'}
buff_a_second {'thinking'}
buff_b_second {'concept', 'abstraction'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'explanation'}
buff_b_first {'idea', 'delay', 'music_genre'}
buff_a_second {'thinking'}
buff_b_second {'expressive_style', 'pause', 'content'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'explanation'}
buff_b_first {'computer_operation', 'activity', 'commercial_document', 'message', 'scu

buff_b_first {'paper', 'walk', 'excavation', 'large_indefinite_quantity', 'platform'}
buff_a_second {'expressive_style', 'pause', 'content'}
buff_b_second {'horizontal_surface', 'travel', 'artifact', 'material', 'indefinite_quantity'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'idea', 'delay', 'music_genre'}
buff_b_first {'growth_rate', 'quantity', 'abstraction'}
buff_a_second {'expressive_style', 'pause', 'content'}
buff_b_second {'entity', 'concept', 'rate'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'idea', 'delay', 'music_genre'}
buff_b_first {'move', 'procedure'}
buff_a_second {'expressive_style', 'pause', 'content'}
buff_b_second {'activity'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'idea', 'delay', 'music_genre'}
buff_b_first {'religion', 'premise', 'machine'}
buff_a_second {'expressive_style', 'pause', 'content'}
buff_b_second {'institution', 'postulate', 'device'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1r

buff_b_first {'explanation'}
buff_a_second {'sameness', 'activity'}
buff_b_second {'thinking'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'procedure', 'use', 'similarity'}
buff_b_first {'rotation', 'generator', 'engineering', 'travel', 'force'}
buff_a_second {'sameness', 'activity'}
buff_b_second {'turning', 'discipline', 'electronic_device', 'physical_phenomenon'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'procedure', 'use', 'similarity'}
buff_b_first {'relation', 'quantity'}
buff_a_second {'sameness', 'activity'}
buff_b_second {'concept', 'abstraction'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'procedure', 'use', 'similarity'}
buff_b_first {'idea', 'delay', 'music_genre'}
buff_a_second {'sameness', 'activity'}
buff_b_second {'expressive_style', 'pause', 'content'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'procedure', 'use', 'similarity'}
buff_b_first {'computer_operation', 'activity', 'co

buff_b_second {'plastic_art', 'movement', 'matter', 'artifact'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'engineering'}
buff_b_first {'explanation'}
buff_a_second {'discipline'}
buff_b_second {'thinking'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'engineering'}
buff_b_first {'relation', 'quantity'}
buff_a_second {'discipline'}
buff_b_second {'concept', 'abstraction'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'engineering'}
buff_b_first {'idea', 'delay', 'music_genre'}
buff_a_second {'discipline'}
buff_b_second {'expressive_style', 'pause', 'content'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'engineering'}
buff_b_first {'computer_operation', 'activity', 'commercial_document', 'message', 'sculpture'}
buff_a_second {'discipline'}
buff_b_second {'operation', 'art', 'act', 'document', 'communication'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'engineering'}
buff_b_first 

buff_b_first {'explanation'}
buff_a_second {'shape', 'check'}
buff_b_second {'thinking'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'solid', 'watch'}
buff_b_first {'rotation', 'generator', 'engineering', 'travel', 'force'}
buff_a_second {'shape', 'check'}
buff_b_second {'turning', 'discipline', 'electronic_device', 'physical_phenomenon'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'solid', 'watch'}
buff_b_first {'relation', 'quantity'}
buff_a_second {'shape', 'check'}
buff_b_second {'concept', 'abstraction'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'solid', 'watch'}
buff_b_first {'idea', 'delay', 'music_genre'}
buff_a_second {'shape', 'check'}
buff_b_second {'expressive_style', 'pause', 'content'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'solid', 'watch'}
buff_b_first {'computer_operation', 'activity', 'commercial_document', 'message', 'sculpture'}
buff_a_second {'shape', 'check'}
buff_b_seco

In [34]:
dist

array([[0.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 0.51851852, 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 0.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 0.80952381, 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 0.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 0.62962963,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 0.        , 0.75757576,
        1.        , 1.        , 0.85964912, 1.        , 1.        ,
        1.        , 1.        , 1.        , 1

In [26]:
buff[0][0]

('much', 'card', 'deck', 'well', 'shuffled')

#### Top ten closest articles with fuzzy metrics of titles

In [37]:
@interact(ind=(0,len(buff)-1,1))
def h(ind=0):
    pp = pprint.PrettyPrinter(indent=4)
    print(' '.join(buff[ind][0]))
    pp.pprint([buff[i][0] for i in dist[ind][:].argsort()[1:11]])


interactive(children=(IntSlider(value=0, description='ind', max=19), Output()), _dom_classes=('widget-interact…

In [38]:
@interact(ind=(0,len(buff)-1,1))
def hypernyms(ind=0):
    pp = pprint.PrettyPrinter(indent=4)
    print(' '.join(buff[ind][0]))
    pp.pprint(buff[ind][1])

interactive(children=(IntSlider(value=0, description='ind', max=19), Output()), _dom_classes=('widget-interact…